<a href="https://colab.research.google.com/github/AntonioDeFA/etl_pyspark/blob/develop/Dados_IBGE_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark findspark
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=92330ceca0ae08ccd232f577f9df7a0bf03d024a4423854442da781a7c108da7
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.9 MB/s eta 0:00:00


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import udf
from pyspark.sql.functions import upper
from pyspark.sql.types import StringType
from unidecode import unidecode

In [3]:
def remove_accents(input_str):
    return unidecode(input_str)

In [4]:
spark = SparkSession.builder \
    .appName("Join and Export to CSV") \
    .getOrCreate()

df_ibge = spark.read.csv('/content/cities-brazil-ibge.csv', header=True, inferSchema=True)
df_cases_covid = spark.read.csv('/content/cases-brazil-cities-time_2022.csv', header=True, inferSchema=True)

In [5]:
df_cases_covid_remove_stats = df_cases_covid.withColumn("city", split(df_cases_covid["city"], "/")[0])

In [6]:
remove_accents_udf = udf(remove_accents, StringType())
df_cases_covid_remove_accent = df_cases_covid_remove_stats.withColumn("city", remove_accents_udf(df_cases_covid_remove_stats["city"]))

In [7]:
df_cases_covid_upper = df_cases_covid_remove_accent.withColumn("city", upper(df_cases_covid_remove_accent["city"]))

In [8]:
joined_df_ibge_covid_cases = df_ibge.withColumnRenamed("LocalCidade", "city").join(df_cases_covid_upper, 'city', 'inner')


In [ ]:
joined_df_ibge_covid_cases_translated = joined_df_ibge_covid_cases.withColumnRenamed("epi_week ", "epi_mes") \
.withColumnRenamed("date", "data") \
.withColumnRenamed("state", "estado") \
.withColumnRenamed("city", "cidade") \
.withColumnRenamed("newDeaths", "novas_mortes") \
.withColumnRenamed("deaths", "mortes") \
.withColumnRenamed("newCases", "novos_casos") \
.withColumnRenamed("totalCases", "total_casos") \
.withColumnRenamed("deaths_per_100k_inhabitants", "morte_por_100k_habitantes") \
.withColumnRenamed("totalCases_per_100k_inhabitants", "total_morte_por_100k_habitantes") \
.withColumnRenamed("deaths_by_totalCases", "mortes_por_total_de_casos")

In [19]:
joined_df_ibge_covid_cases_translated = joined_df_ibge_covid_cases
old_names = ["epi_week",
            "date",
            "state",
            "city",
            "newDeaths",
            "deaths",
            "newCases",
            "totalCases",
            "deaths_per_100k_inhabitants",
            "totalCases_per_100k_inhabitants",
            "deaths_by_totalCases"]
new_names = ["epi_mes",
             "data",
             "estado",
             "cidade",
             "novas_mortes",
             "mortes",
             "novos_casos",
             "total_casos",
             "morte_por_100k_habitantes",
             "total_morte_por_100k_habitantes",
             "mortes_por_total_de_casos"]
for i in range(len(old_names)):
  joined_df_ibge_covid_cases_translated = \
    joined_df_ibge_covid_cases_translated. \
      withColumnRenamed(old_names[i],
                        new_names[i])

In [20]:
joined_df_ibge_covid_cases_translated.show()

+-------------------+--------+-------+-------------------+------------+--------+---------+-------------------+----------------+-------------+-----+-----------------------+-----------------------+--------+-------+----------+------+-------+------------+------+-----------+-----------+-------------------------+-------------------------------+-------------------------+
|             cidade|IBGECode|LocalUF|        LocalEstado|RegiaoBrasil|Latitude|Longitude|          Gentilico|PopEstimada_2018|PopCenso 2010| IDHM|ReceitasRealizadas_2014|DespesasEmpenhadas_2014|Pib_2014|epi_mes|      data|estado| ibgeID|novas_mortes|mortes|novos_casos|total_casos|morte_por_100k_habitantes|total_morte_por_100k_habitantes|mortes_por_total_de_casos|
+-------------------+--------+-------+-------------------+------------+--------+---------+-------------------+----------------+-------------+-----+-----------------------+-----------------------+--------+-------+----------+------+-------+------------+------+--------

In [ ]:
joined_df_ibge_covid_cases_droped_columns = joined_df_ibge_covid_cases_translated.drop("epi_week","RegiaoBrasil")

In [ ]:
joined_df_ibge_covid_cases_droped_columns.write.csv("relatorio-final.csv", header=True)

spark.stop()